In [1]:
import pandas as pd
import numpy as np

In [180]:
df = pd.read_csv('./data_down/1223_fifo_2wk.csv', sep = '\001')
df.columns = pd.Series(df.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
df2 =  pd.read_csv('./data_down/fifo_out_2w.csv', sep = '\001')
df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
# df = df2

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19428/3058550165.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = pd.Series(df.columns).str.replace('.+\.', '')
C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_19428/3058550165.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')


In [181]:
def data_(stats):
    fefo = df[df['fifo_fefo'] == stats].sort_values(['sku' , 'received_date' ], ascending=True)
    fefo = fefo[fefo['week_7']!=0]
    fefo['out_amt'] = fefo['week_6'] - fefo['week_7']


    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id',   'fifo_fefo', 'start_of_week', 'end_of_week']
        )['sku'].transform('count')
    fefo = fefo[fefo['flag'] != 1]

    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id',   'fifo_fefo', 'start_of_week', 'end_of_week']
        )['out_amt'].transform(min)
    fefo = fefo[fefo['flag'] > 0]


    fefo.tail(8)
    return fefo

def data2_(stats, df):
    fefo = df
    if stats == 'fifo':
        fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
    else:
        fefo = fefo.sort_values(['sku' , 'received_date'], ascending=False)
    fefo_out = fefo.groupby(
        ['ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week', 'sku'] , as_index= False
        ).agg(
            old_remain_amt = ('week_7', 'first'), # old remain
            old_remain_location = ('location', 'first'), # 易过期
            old_remain_date = ('received_date', 'first'), 
            old_remain_lock_code = ('lock_codes', 'first'), 

            new_export_amt = ('out_amt', 'mean'),  # new export
            new_export_location = ('location', 'last'),
            new_export_date = ('received_date', 'last'), 
            new_export_lock_codes = ('lock_codes', 'first'), 
    )
    return fefo_out


In [121]:
# fefo_out = fefo_out[~fefo_out['old_remain_location'].str.contains(fefo_out['new_export_location'][0])]

In [182]:
fifo = data_('fifo')
fefo = data_('fefo')
# fifo.head(5).sort_values(['sku' , 'received_date'], ascending=False)

In [183]:
pd.concat([
    data2_('fefo', fefo), data2_('fifo', fifo)
    ], axis = 0)\
        .to_csv('./data_up/fifo_fefo_alert_thur.csv', index = None, encoding = 'utf_8_sig')

In [144]:
data2_('fifo', fifo)

,ou_code,wms_warehouse_id,fifo_fefo,start_of_week,end_of_week,sku,old_remain_amt,old_remain_location,old_remain_date,old_remain_lock_code,new_export_amt,new_export_location,new_export_date,new_export_lock_codes
0,COSTASHHTS,COSTA_SH,fifo,20211210,20211217,COSTA050466,494.0,D-06-03-01-A,2021-11-22,Available,23.50,"SHIP-01,D-06-12-01",2021-10-28,Available
1,COSTASHHTS,COSTA_SH,fifo,20211210,20211217,COSTA060623,9.0,D-06-08-03,2021-11-04,Available,4.00,"D-04-20-05,SHIP-01",2021-10-21,Available
2,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,6ZD29AA,1804.0,"12-07-2,12-12-2",2021-12-07,1001-34,652.00,"22-09-A,13-07-3,11-04-3",2021-11-30,1001-34
3,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,C2P24AA,57.5,P-04-01-1,2021-12-09,1001-34,27.25,X-02-08-3,2021-11-24,1001-34
4,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,CB382A,11.0,"B-0086,B-0116",2021-06-28,1001-34,3.00,"B-0115,B-0116,B-0086",2021-06-06,1001-34
5,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,CE320A,66.5,01-07-1,2021-11-24,1001-34,39.00,P-08-09-2,2021-06-18,1001-34
6,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,CF226XC,267.0,"P-09-01-1,P-06-23-1",2021-10-21,1002-34,45.75,23-08-3,2021-10-19,1002-34
7,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,CN624AA,176.0,20-06-3,2021-11-19,1001-34,18.00,SMALL-51,2021-08-25,1001-34
8,HPPXXSHMGS,HPI_SH,fifo,20211210,20211217,W1A57A#AB2,12.0,R4345,2021-12-06,1001-34,124.00,R4749,2021-06-09,1001-34
9,HPPXXWHWDS,HPI_WH,fifo,20211210,20211217,8PE95A#AB2,9.0,"A-10-03-1,A-10-01-1,A-10-02-1",2021-12-08,1001-34,2.50,"B-11-06-1,B-12-24-1,B-12-19-1",2021-12-03,1001-34
